# Reissner-Mindlin Plate

Find $(w,\beta)\in H_{\Gamma_D}^1(\Omega)\times [H_{\Gamma_D}^1(\Omega)]^2$ such that for all $(\delta w,\delta \beta)\in H_{\Gamma_D}^1(\Omega)\times [H_{\Gamma_D}^1(\Omega)]^2$

$$
\begin{aligned}
\frac{1}{12}\int_{\Omega}\epsilon(\beta):\epsilon(\delta\beta)\,dx + \frac{1}{2t^2}\int_{\Omega}(\nabla w-\beta)\cdot(\nabla\delta w-\delta\beta)\,dx& = \int_{\Omega}f_z\delta w\,dx && \text { in  } \Omega,
\\
w & = 0 
&& \text{ on } \Gamma_D,\\
\beta & = 0 
&& \text{ on } \Gamma_D.
\end{aligned}
$$

#### Import NGSolve and Netgen Python modules:

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw

#### Parameter

In [ ]:
order = 2
maxh = 0.5
# force
fz = 0.01
# thickness
t = 1

#### Mesh

In [ ]:
R = 5
from netgen.geom2d import SplineGeometry
geo = SplineGeometry()
geo.AddCircle((0,0), R, bc="circ")

mesh = Mesh(geo.GenerateMesh(maxh=maxh))
mesh.Curve(order)
Draw(mesh)

#### Reference solution


In [ ]:
r = sqrt(x**2+y**2)
xi = r/R
w_ex = -3*fz*R**4/16*(1-xi**2)*( (1-xi**2) + 8*(t/R)**2/3)
Draw(w_ex, mesh, "w_ex",deformation=True)

#### Finite element spaces

In [ ]:
fesW = H1(mesh, order=order, dirichlet="circ")
fesB = VectorH1(mesh, order=order, dirichlet="circ")
fes = fesW*fesB
(w,beta), (dw,dbeta) = fes.TnT()

#### Define and assemble linear and bilinear forms

In [ ]:
a = BilinearForm(fes, symmetric=True)
a += 1/12*InnerProduct(Sym(grad(beta)),Sym(grad(dbeta)))*dx
a += 1/(2*t**2)*InnerProduct( grad(w)-beta, grad(dw)-dbeta )*dx
a.Assemble()

f = LinearForm(fes)
f += -fz*dw*dx
f.Assemble()

#### Solve system and draw solution

In [ ]:
gfsol = GridFunction(fes)
gfw, gfbeta= gfsol.components

gfsol.vec.data = a.mat.Inverse(fes.FreeDofs())*f.vec
Draw(gfw, mesh, "w", deformation=True)
Draw(gfbeta, mesh, "beta")

#### Compute error

In [ ]:
norm_w = sqrt(Integrate(w_ex*w_ex, mesh))
l2err = sqrt(Integrate((gfw-w_ex)*(gfw-w_ex), mesh))/norm_w
print(l2err)